# Organizing and Visualizing Datas - 1. Subway Usage for Each Station in Seoul

In [1]:
# Import all the necessary liabraries
import types
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np 

!pip install geopy
from geopy.geocoders import Nominatim 

!pip install folium
import folium 
import branca
print('Libraries imported.')


Libraries imported.


## 1. Organize the first data : Monthly Subway Usage for Each Station(October 2019)

In [2]:
df = pd.read_csv('Board_Alight.csv')
df.head()

,사용일자,노선명,역ID,역명,승차총승객수,하차총승객수,등록일자
0,20191001,일산선,1957,주엽,11535,12035,20191004
1,20191001,일산선,1956,정발산,9430,9681,20191004
2,20191001,일산선,1955,마두,9659,10149,20191004
3,20191001,일산선,1954,백석,12782,12679,20191004
4,20191001,일산선,1953,대곡,2107,1542,20191004


In [3]:
#Remove unnecessary columns and translate the column names
df = df[['역ID','역명','하차총승객수']]
df.columns = ['Sta_ID','Sta_Name','Alight_Num']
print(df.shape)
df.head()

(18320, 3)


,Sta_ID,Sta_Name,Alight_Num
0,1957,주엽,12035
1,1956,정발산,9681
2,1955,마두,10149
3,1954,백석,12679
4,1953,대곡,1542


In [4]:
#Split the dataframe in two, see the document for more
#Df1
df_subway= df[['Sta_ID','Sta_Name']]
print(df_subway.shape)
df_subway.head()

(18320, 2)


,Sta_ID,Sta_Name
0,1957,주엽
1,1956,정발산
2,1955,마두
3,1954,백석
4,1953,대곡


In [5]:
#Df2
df_subgroup = df[['Sta_Name','Alight_Num']]

#Group and sum to get the total number on October 2019
df_subgroup = df_subgroup.groupby(['Sta_Name']).sum()
df_subgroup.reset_index(inplace=True)
print(df_subgroup.shape)
df_subgroup.head()

(509, 2)


,Sta_Name,Alight_Num
0,4.19민주묘지,108788
1,가능,219028
2,가락시장,589718
3,가산디지털단지,1834090
4,가양,648778


In [6]:
#Group Df1 as well
df_subway = df_subway.groupby(['Sta_ID','Sta_Name']).sum()
df_subway.reset_index(inplace=True)
print(df_subway.shape)
df_subway.head()

(598, 2)


,Sta_ID,Sta_Name
0,150,서울역
1,151,시청
2,152,종각
3,153,종로3가
4,154,종로5가


In [7]:
#Drop duplicate stations to get rid of multiple Station IDs for one Station
df_subway.drop_duplicates(['Sta_Name'], keep='first',inplace=True)
print(df_subway.shape)
df_subway.head()

(509, 2)


,Sta_ID,Sta_Name
0,150,서울역
1,151,시청
2,152,종각
3,153,종로3가
4,154,종로5가


In [8]:
#Join Df1 and Df2 on Station Name
df_subway = df_subway.join(df_subgroup.set_index('Sta_Name'), on='Sta_Name')
print(df_subway.shape)
df_subway.head()

(509, 3)


,Sta_ID,Sta_Name,Alight_Num
0,150,서울역,3110466
1,151,시청,1759078
2,152,종각,1361926
3,153,종로3가,1979876
4,154,종로5가,874720


## 2. Organize the second data : Subway Station Coordinates for Each Station ID

In [9]:
df2 = pd.read_csv('Station_Coor.csv')
df2.head()

,전철역코드,전철역명,호선,외부코드,사이버스테이션,X좌표,Y좌표,X좌표(WGS),Y좌표(WGS)
0,2818,가락시장,8,817,2818,525992.0,1108579.0,37.492522,127.118234
1,340,가락시장,3,350,2818,525992.0,1108579.0,37.492522,127.118234
2,2535,종로3가,5,534,153,498060.0,1130332.0,37.571607,126.991806
3,319,종로3가,3,329,153,498060.0,1130332.0,37.571607,126.991806
4,153,종로3가,1,130,153,498060.0,1130332.0,37.571607,126.991806


In [10]:
#Remove unnecessary columns and translate the column names
df_subxy= df2[['전철역코드','전철역명','X좌표(WGS)','Y좌표(WGS)']]
df_subxy.columns=['Sta_ID','Sta_Name_','X_Coor','Y_Coor']

In [11]:
#Sort by Station ID and drop the duplicates to make sure it dropped the same duplicate as df_subway
df_subxy.sort_values(by=['Sta_ID'], inplace=True)
df_subxy.drop_duplicates(['Sta_ID'], keep='first',inplace=True)
print(df_subxy.shape)
df_subxy.head()

(694, 4)


,Sta_ID,Sta_Name_,X_Coor,Y_Coor
296,150,서울,37.554648,126.972559
26,151,시청,37.564718,126.977108
239,152,종각,37.570161,126.982923
4,153,종로3가,37.571607,126.991806
240,154,종로5가,37.570926,127.001849


## 3. Join the first and the second data

In [12]:
#Join On Station ID
df_subway = df_subway.join(df_subxy.set_index('Sta_ID'), on='Sta_ID')

In [13]:
#Drop NaN Values.
df_subway.dropna(inplace=True)
print(df_subway.shape)
df_subway.head()

(487, 6)


,Sta_ID,Sta_Name,Alight_Num,Sta_Name_,X_Coor,Y_Coor
0,150,서울역,3110466,서울,37.554648,126.972559
1,151,시청,1759078,시청,37.564718,126.977108
2,152,종각,1361926,종각,37.570161,126.982923
3,153,종로3가,1979876,종로3가,37.571607,126.991806
4,154,종로5가,874720,종로5가,37.570926,127.001849


In [14]:
#Values for Sta_Name_ are more consistant so drop Sta_Name
df_subway.drop(columns=['Sta_Name'],inplace=True)
print(df_subway.shape)
df_subway.head()

(487, 5)


,Sta_ID,Alight_Num,Sta_Name_,X_Coor,Y_Coor
0,150,3110466,서울,37.554648,126.972559
1,151,1759078,시청,37.564718,126.977108
2,152,1361926,종각,37.570161,126.982923
3,153,1979876,종로3가,37.571607,126.991806
4,154,874720,종로5가,37.570926,127.001849


In [15]:
#Store the data
data_subway = df_subway
%store data_subway
del data_subway

Stored 'data_subway' (DataFrame)


# 4. Visualize the data

In [17]:
#Create scale for color
threshold_scale = np.linspace(df_subway['Alight_Num'].min(),
                              df_subway['Alight_Num'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist()
threshold_scale

[1960, 648065, 1294170, 1940275, 2586380, 3232486]

In [20]:
#Create Map
map_seoul = folium.Map(location=[37.5666791, 126.9782914],zoom_start=12,tiles='Stamen Toner') #Seoul's coordinate

for lat, lng, label, pop in zip(df_subway['X_Coor'], df_subway['Y_Coor'], df_subway['Sta_Name_'], df_subway['Alight_Num']):
    label = folium.Popup(label, parse_html=True)
    if pop in range(threshold_scale[0], threshold_scale[1]):
        color = '#ff0000'
    elif pop in range(threshold_scale[1], threshold_scale[2]):
        color = '#f6ff00'
    elif pop in range(threshold_scale[2], threshold_scale[3]):
        color = '#00ff08'
    elif pop in range(threshold_scale[3], threshold_scale[4]):
        color = '#0d00ff'
    elif pop in range(threshold_scale[4], threshold_scale[5]+1):
        color = '#ff00f7'
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#000000',
        fill_opacity=1,
        parse_html=False).add_to(map_seoul)  

In [21]:
#For Map Legend
legend_html = '''
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    top: 0px;
    right: -100px;
    width: 350px;
    height: 80px;
    z-index:9999;
    font-size:14px;
    ">
    <p></a>&emsp;Number of Alights (Oct 2019)</p>
    <p><a style="color:#ff0000;font-size:150%;margin-left:20px;">●</a>&emsp;Very Low Traffic</p>
    <p><a style="color:#f6ff00;font-size:150%;margin-left:20px;">●</a>&emsp;Low Traffic</p>
    <p><a style="color:#00ff08;font-size:150%;margin-left:20px;">●</a>&emsp;Medium Traffic</p>
    <p><a style="color:#0d00ff;font-size:150%;margin-left:20px;">●</a>&emsp;High Traffic</p>
    <p><a style="color:#ff00f7;font-size:150%;margin-left:20px;">●</a>&emsp;Very High Traffic</p>
</div>
<div style="
    position: fixed; 
    top: 0px;
    right: 0px;
    width: 250px;
    height: 220px; 
    z-index:9998;
    font-size:14px;
    background-color: #E0FFFF;
    opacity: 0.8;
    ">
</div>
{% endmacro %}
'''

legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)

map_seoul.get_root().add_child(legend)